In [23]:
# Check version protobuf and generate protobuf file

!protoc --version
!protoc --python_out=. 'proto/transaction_product.proto'

libprotoc 3.21.12


In [24]:
# Install library

!pip install faker-commerce

In [ ]:
# Import library

import os
import uuid
import faker_commerce
from datetime import datetime, timedelta
from dotenv import load_dotenv
from faker import Faker
from kafka import KafkaProducer
from pathlib import Path
from proto import transaction_product_pb2
from time import sleep

In [2]:
# Load environment file

dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [ ]:
# Set kafka variable configuration

kafka_host = os.getenv('KAFKA_HOST')
kafka_host_broker_2 = os.getenv('KAFKA_BROKER_2_HOST')
kafka_topic = os.getenv('topic')
kafka_topic_partition = os.getenv('topic') + "-1"

dataeng-kafka
dataeng-broker-kafka
assignment
assignment-1
2


In [4]:
# Set faker for dummy data add ecommerce to faker

faker = Faker()
faker.add_provider(faker_commerce.Provider)

In [ ]:
# Create Faker data with format protobuf

class Datagenerator(object):
    @staticmethod
    def get_data():

        # Random time stamp for random transaction range 5 second
        timestamp = datetime.now() - timedelta(seconds=faker.random_int(min=0, max=5))
        transaction = transaction_product_pb2.TransactionProduct()
        transaction.id = uuid.uuid4().__str__()
        transaction.product_name = faker.ecommerce_name()
        transaction.customer_name = faker.name()
        transaction.status = faker.random_element(elements=("checkout", "cancel"))
        transaction.timestamp = datetime.strftime(timestamp, "%d-%m-%y %H:%M:%S")
        transaction.total_payment = faker.random_int(min=5000, max=5000000)
        return transaction

In [ ]:
# Create producer kafka using 2 broker with 2 replication

producer = KafkaProducer(bootstrap_servers=[f'{kafka_host}:9092', f'{kafka_host_broker_2}:9095'], value_serializer=lambda v: v.SerializeToString())

dataeng-kafka
dataeng-broker-kafka


NoBrokersAvailable: NoBrokersAvailable

In [ ]:
# Send message using topic assignment with 4 partition

for i in range(0, 200):
    transaction_data = Datagenerator.get_data()
    response = producer.send(topic=kafka_topic, value=transaction_data)
    print(transaction_data)
    sleep(5)

NameError: name 'producer' is not defined